In [ ]:
import pymc3 as pm
import theano.tensor as tt
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import arviz as az
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats

%config InlineBackend.figure_formats = ['retina']

np.random.seed(100)

In [ ]:
sup_yield      = [.9, .5, .8] # unkown in the real world
sup_yield_sd   = [.1, .2, .2] # unkown in the real world
prices         = [220, 100, 120] # known
max_order_size = [100, 80, 100] # known

n_obs_sup      = [30, 20, 2] # #times we have ordered from each supplier

In [ ]:
data = []
for sy, sy_sd, obs in zip(sup_yield, sup_yield_sd, n_obs_sup):
    data.append(
        pm.Beta.dist(mu=sy, sd=sy_sd, shape=obs).random()
    )
data

In [ ]:
df = pd.DataFrame(data).T
df = df.unstack().to_frame('yield')
df.index = df.index.set_names(['supplier', 'obs'])
g = sns.FacetGrid(data=df.reset_index().dropna(), col='supplier')
g.map(sns.histplot, 'yield', kde=False)

In [ ]:
sales_price   = 500
holding_cost = 100 # storage

In [ ]:
@np.vectorize
def loss(in_stock, demand, buy_price, 
         sales_price=sales_price, holding_cost=holding_cost):
    # how much do we earn per lunch
    margin = sales_price - buy_price
    
    # do we have more in stock than demanded?
    if in_stock > demand:
        total_profit = demand * margin
        
        # everything else goes to holding
        total_holding_cost = (in_stock - demand) * holding_cost
        
        reward = total_profit - total_holding_cost
    else:
        reward = in_stock * margin
        
    # we minimize
    return (-1) * reward

In [ ]:
plt.figure(figsize=(16, 8))
in_stock = np.linspace(0, 100, 200)
plt.plot(in_stock, -loss(in_stock, 50, 50), lw=5)
plt.axvline(50, c='k', ls='--', label='assumed demand')
plt.xlabel('in stock')
plt.ylabel('profit (neg loss)')
sns.despine()
plt.legend();

In [ ]:
demand_samples = stats.poisson(60, 40).rvs(1000)
sns.displot(demand_samples, kde=True)

In [ ]:
plt.figure(figsize=(16, 8))
plt.scatter(demand_samples, -loss(100, demand_samples, buy_price=10), s=20)
plt.axvline(100, c='k', ls='--', label='assumed in_stock')
plt.xlabel('demand')
plt.ylabel('profit (neg loss)')
sns.despine()
plt.legend();

## PyMC model

In [ ]:
data

In [ ]:
with pm.Model() as model:
    # priors for each supplier
    a = pm.HalfNormal('alfa', sd=10., shape=3) + 1
    b = pm.HalfNormal('beta', sd=10., shape=3) + 1
    
    # different likelehood for each supplier
    # because we have different n_obs
    for i, d in enumerate(data):
        pm.Beta(f"supplier_yield_obs_{i}", 
                   alpha=a[i], beta=b[i],
                   observed=d
               )
        
    trace = pm.sample(1000, return_inferencedata=True)

In [ ]:
# test convergence
az.plot_energy(trace)

Generate future scenarios

In [ ]:
with model:
    post_pred = pm.sample_posterior_predictive(trace, 1000)

In [ ]:
sup_yield_post = pd.DataFrame({
    k: v[:, 1] for k, v in post_pred.items()
})
sup_yield_post_tidy = sup_yield_post.unstack().to_frame('yield')
sup_yield_post_tidy.index = sup_yield_post_tidy.index.set_names(['supplier', 'obs'])
g = sns.FacetGrid(data=sup_yield_post_tidy.reset_index().dropna(), col='supplier')
g.map(sns.histplot, 'yield', kde=False)

In [ ]:
def calc_yield_and_price(orders, 
                        in_sup_yield=np.array([.9, .5, .8]),
                        prices=prices
                        ):
    orders = np.asarray(orders)
    
    full_yield = np.sum(in_sup_yield * orders)
    prices_per_item = np.sum(orders * prices) / np.sum(orders)
    
    return full_yield, prices_per_item

calc_yield_and_price([100, 60, 60])

In [ ]:
def objective(orders,
              in_sup_yield=sup_yield_post,
              demand_sample=demand_samples,
              max_order_size=max_order_size):
        
    orders = np.asarray(orders)
    losses = []
    
    # negative orders impossible
    if np.any(orders < 0):
        return np.inf
    
    # ordering more than the supplier can ship impossible
    if np.any(orders > max_order_size):
        return np.inf

    for i, sy in in_sup_yield.iterrows():   
        full_yield, pp_item = calc_yield_and_price(
            orders,
            in_sup_yield=sy
        )
        
        # evaluate loss over each sample with one from the demand
        # distribution
        loss_i = loss(full_yield, demand_samples[i], pp_item)
        
        losses.append(loss_i)
        
    return np.array(losses)

In [ ]:
from scipy import optimize

In [ ]:
bounds = [(0, max_order) for max_order in max_order_size]
start_value = [50, 50, 50]

In [ ]:
opt = optimize.minimize(lambda *args: np.mean(objective(*args)),
                           start_value,
                           bounds=bounds
                       )

In [ ]:
# optimal order
np.ceil(opt.x)

Evaluation

In [ ]:
supplier_yield_mean = pd.DataFrame([np.mean(d) for d in data]).T
supplier_yield_mean

In [ ]:
 objective(start_value, in_sup_yield=supplier_yield_mean,
              demand_sample=[100])

In [ ]:
opt_non_stoch = optimize.minimize(lambda *args: 
    objective(*args, in_sup_yield=supplier_yield_mean,
              demand_sample=[100]),
                           start_value,
                           bounds=bounds
                       )

In [ ]:
np.ceil(opt_non_stoch.x)

In [ ]:
np.random.seed(123)

data_new = []

for sy, sy_sd, obs in zip(sup_yield, sup_yield_sd, n_obs_sup):
    data_new.append(
        pm.Beta.dist(mu=sy, sd=sy_sd, shape=1000).random()
    )
    
data_new = pd.DataFrame(data_new).T
data_new.head().add_prefix('Supplier ')

In [ ]:
neg_loss_stoch = -objective(opt.x, in_sup_yield=data_new) / demand_samples
neg_loss_non_stoch = -objective(opt_non_stoch.x, in_sup_yield=data_new) / demand_samples

ax = sns.histplot(neg_loss_stoch, label='stochastic', kde=False)
plt.axvline(np.mean(neg_loss_stoch), label='expected stochastic')


sns.histplot(neg_loss_non_stoch, label='non stochastic', color='orange', kde=False, ax=ax)
plt.axvline(np.mean(neg_loss_non_stoch), label='expected non-stochastic', color='orange')

plt.legend()

plt.xlabel('Profit (negative loss)')
plt.ylabel('Occurances');